# Addiction Simulator

## Setup Virtual Environment
1. make sure python3 is installed on your system.
2. make sure you have a python virtual environment setup: 
python -m pip install --upgrade pip setuptools virtualenv
3. create a python environment: 
python -m venv venv (this creates a virtual environment called venv)
4. if applicable add /venv/ to your .gitignore.
5. activate the virtual environment: 
either \venv\Scripts\activate.bat on windows 
or source venv/bin/activate on mac+linux


## Import Libraries

In [12]:
#basic
from os import system, name
import random
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import datetime

#pytorch for gpu processing of ML model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

#rich library for Terminal UI
from rich.jupyter import print
#from rich import print
from rich.prompt import IntPrompt


#hide pytorch warnings (should eventually be resolved)
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output

## Agent's Brain (Deep Q-Learning)

### Neural Network Model

In [13]:
class Network(nn.Module):  
    def __init__(self, input_size, nb_action):
        #ref: https://discuss.pytorch.org/t/super-model-in-init/97426
        #super(Network, self).__init__()
        super().__init__() #pytorch's NN model
        self.input_size = input_size
        self.nb_action = nb_action
        self.fc1 = nn.Linear(input_size, 30)#arbitrarily chose 30 hidden layers
        self.fc2 = nn.Linear(30, nb_action)
    
    #base pytorch NN model runs and we override the
    #forward function with our own relu activation function
    def forward(self, state):
        x = F.relu(self.fc1(state))
        q_values = self.fc2(x)
        return q_values

### Experience Replay Model
This model is used for training our DQN model. It stores the transitions that the agent observes, allowing us to reuse this data later. By sampling from it randomly, the transitions that build up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the DQN training procedure.

In [14]:
class ReplayMemory(): 
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
    
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
    
    def sample(self, batch_size):
        samples = zip(*random.sample(self.memory, batch_size))
        return map(lambda x: Variable(torch.cat(x, 0)), samples)

### DQN Ensemble
Comprised of a neural network model and a memory model. 
* The NN takes in observation of sensor data (brain chemicals) and chooses actions based on the relu activation function. 
* The agent will sample some of the sensor data and store in long term memory to be reused later for training. 
* We also use the Adam Optimisation algorithm. This is an extension to stocastic gradient desent to update weights of the neural network. 

In [20]:
class Dqn():
    def __init__(self, input_size, nb_action, gamma):
        self.gamma = gamma
        self.reward_window = []
        self.model = Network(input_size, nb_action)
        self.memory = ReplayMemory(100000)
        self.optimizer = optim.Adam(self.model.parameters(), lr = 0.001)
        self.last_state = torch.Tensor(input_size).unsqueeze(0)
        self.last_action = 0
        self.last_reward = 0
    
    # select action for x duration
    def select_action(self, state):
        #softmax converts numbers into probabilities
        #Q values are the output of the neural network
            #view q values
        q_value_tensor = self.model(Variable(state, volatile = True)) 
        q_values = [q_value.detach().numpy() for q_value in q_value_tensor]
            #print(q_values)
            #viz q value for each action, (T value by user choice)
            #pie chart 0/1 #seperate action
        # Temperature value = 100. closer to zero the less sure the NN will be to taking the action
        probs = F.softmax(self.model(Variable(state, volatile = True))*100) # T=100
        action_prob = [prob.detach().numpy() for prob in probs]

        action = probs.multinomial(num_samples=1) # action taken
        #q_values[0][action] #qualiy of taking action in state
        #action_prob[0][action] #probability of taking action

        #return the action taken, q values and probabilities of taking action given state.
        return action.data[0,0], q_values[0][action], action_prob[0][action]
        #return action.data[0,0], q_values, action_prob
    
    #to train our AI
    #forward propagation then backproagation
    # get our output, target, compare our output to the target to compute the loss error
    # backproagate loss error into the nn and use stochastic gradient descent we update the weights according to how much they contributed to the loss error
    def learn(self, batch_state, batch_next_state, batch_reward, batch_action):
        outputs = self.model(batch_state).gather(1, batch_action.unsqueeze(1)).squeeze(1)
        next_outputs = self.model(batch_next_state).detach().max(1)[0]
        target = self.gamma*next_outputs + batch_reward
        td_loss = F.smooth_l1_loss(outputs, target)
        self.optimizer.zero_grad()
        td_loss.backward(retain_graph = True)
        self.optimizer.step()
    
    #When ai reaches a new state we update everything
    #update action, last action becomes the new action but also the last state becomes the new state and last reward becomes the new state
    # we then get this new transition and update our reward window to track training progress and exploration
    def update(self, reward, new_signal):
        new_state = torch.Tensor(new_signal).float().unsqueeze(0)
        self.memory.push((self.last_state, new_state, torch.LongTensor([int(self.last_action)]), torch.Tensor([self.last_reward])))
        action, q, p = self.select_action(new_state)
        if len(self.memory.memory) > 100:
            batch_state, batch_next_state, batch_action, batch_reward = self.memory.sample(100)
            self.learn(batch_state, batch_next_state, batch_reward, batch_action)
        self.last_action = action
        self.last_state = new_state
        self.last_reward = reward
        self.reward_window.append(reward)
        #if len(self.reward_window) > 1000:
        #    del self.reward_window[0]
        return action, q, p
    
    def score(self):
        #return sum(self.reward_window)/(len(self.reward_window)+1.0)
        return sum(self.reward_window)
    
    def save(self):
        torch.save({'state_dict': self.model.state_dict(),
                    'optimizer' : self.optimizer.state_dict(),
                   }, 'last_brain.pth')
    def load(self):
        if os.path.isfile('last_brain.pth'):
            print("=> loading checkpoint... ")
            checkpoint = torch.load('last_brain.pth')
            self.model.load_state_dict(checkpoint['state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            print("done !")
        else:
            print("no checkpoint found...")

## Simulation

In [21]:
class Simulation():    
    def __init__(self):
        # Agent Brain - a neural network that represents our Q-function
        self.agent = Dqn(2,3,0.9) # 4 sensors, 8 actions, gama = 0.9
        #Agent's Actions
        #self.actions = ['Sleep', 'Binge on Internet', 'Work', 'Exercise', 'Socialise', 'Drink Alcohol', 'Smoke', 'Take Cocaine'] #8 actions
        self.actions = ['Do Nothing', 'In-Active Lever', 'Active Lever']
        
        # Mehdi Keramati's definition of an animal ref Mehdi Keramati's Homeostatic RL sim of addiction
        ## fatigue or lever/action cost
        self.fatigue = -1

        #Keramati's outcome, e.g cocaine = 50, representing the dose of self-administered drug,
        self.outcome = [0,0,50] #all others were scaled from cocaine as their assumed impact on the brain. #[0,10,5,12.5,12.5,10,25,50]#self.outcome = [0,0,0,0,0,0,0,50] #all others were scaled from cocaine as their assumed impact on the brain. #[0,10,5,12.5,12.5,10,25,50]
        self.outcomeBuffer = 0
        self.epochs_inactive = 4 # python index starts at 0 so (1+4)* 4 = 5*4 = 20sec lever will be active for 4 seconds then disabled for 20secs, on the 20th second it will be active.
        self.outcome_to_disable = []

        ## Homeostatic System
        self.initialInState = 0
        self.initialSetpoint = 200
        self.inStateLowerBound = 0
        self.outcomeDegradationRate = 0.007 # e.g dose of cocaine that the animal loses in every time-step
        self.outcomeAbsorptionRatio = 0.12 # e.g proportion of the injected cocaine that affects the brain right after infusion
        self.estimatedNonHomeostaticReward = 0.0

        ## Allostatic (Stress) System
        self.setpointShiftRate = 0.0018
        self.setpointRecoveryRate = 0.00016
        self.optimalInStateLowerBound = 100
        self.optimalInStateUpperBound = 200

        ## Drive Function
        self.m = 3 # Parameter of the drive function : m-th root
        self.n = 4 # Parameter of the drive function : n-th pawer
        self.driveReductionReward = 0.0

        ## Goal-directed system
        self.updateRewardRate = 0.2  # Learning rate for updating the non-homeostatic reward function
        self.updateOutcomeRate = 0.2  # Learning rate for updating the outcome function
        
        #agent's sensors or observation space
        self.last_action = 0 #c(external state /exited)
        self.internal_state = float(self.initialInState) #internal variable that moves homeostatic setpoint
        self.setpoint_S = float(self.initialSetpoint) #homeostatic setpoint

        
        # the mean score curve (sliding window of the rewards) with 
        # respect to time.
        self.total_reward = []

        #Environment: the agent's grid world is elapsing time from 1 to endpoint
        self.total_time = 4 # total time in hours (size of the world) e.g 1 year or 8760hrs, or 4 (1hr pretraining and 3 hours seeking cocaine)
        self.total_epochs = (self.total_time*3600)/4 #e.g 20 secs is 5 trials or epochs of time or 1 hour = 3600secs/4secs = 900 epochs, 8760hrs is 7,884,000 epochs

        self.current_epoch = 0 # e.g 1=4secs

        # reward agent wants to maximise this will be the homeostatic reward
        self.reward_received = 0.0

        #create data table
        #self.df = pd.DataFrame(columns=['epoch', 'action', 'q_value_0', 'q_value_1','q_value_2','t_prob_0', 't_prob_1','t_prob_2', 'epochs_inactive', 'internal_variable', 'homeostatic_setpoint', 'reward', 'score'])

        self.finish = False #trigger to end simulation
        while not self.finish:
            self.finish = self.next_time_interval()# begin simulation and repeat until triggered not to
        if(self.finish):
            #print("saving brain...")
            #brain.save()
            plt.title("Rewards")
            plt.xlabel("Epoch")
            plt.ylabel("Reward")
            plt.plot(self.total_reward)
            plt.show()

            plt.title("Internal_variable [Dopamine]")
            plt.xlabel("Epoch")
            plt.ylabel("Internal Variable")
            plt.plot(self.df["internal_variable"])
            plt.show()

            plt.title("Homeostatic Setpoint")
            plt.xlabel("Epoch")
            plt.ylabel("Homeostatic Setpoint")
            plt.plot(self.df["homeostatic_setpoint"])
            plt.show()

    def next_time_interval(self):
        ## get the time left
        epochs_left = int(self.total_epochs - self.current_epoch)
        
        ## agent input state vector, composed of the five brain signals or observations received from being in the environment
        current_state = [self.internal_state, self.setpoint_S] #self.last_action,, self.current_epoch
        action_to_take, qValues, transitionProbs = self.agent.update(self.reward_received, current_state) # playing the action from the ai (dqn class)
        self.total_reward.append(self.agent.score())
        
        #we can take the agent's NN and call forward to output via softmax q values for each state.
        suggested_action = self.actions[action_to_take]
        #sa = int(0 if suggested_action=="Sleep" else 1 if suggested_action=="Binge on Internet" else 2 if suggested_action=="Work" else 3 if suggested_action=="Exercise" else 4 if suggested_action=="Socialise" else 5 if suggested_action=="Drink Alcohol" else 6 if suggested_action=="Smoke" else 7 if suggested_action=="Take Cocaine" else -1)
        sa = int(0 if suggested_action=="Do Nothing" else 1 if suggested_action=="In-Active Lever" else 2 if suggested_action=="Active Lever" else -1)
        #give user options
        #print("0. [bold dark_violet]Sleep[/bold dark_violet]\n1. [bold dark_violet]Binge on Internet[/bold dark_violet]\n2. [bold dark_violet]Work[/bold dark_violet]\n3. [bold dark_violet]Exercise[/bold dark_violet]\n4. [bold dark_violet]Socialise[/bold dark_violet]\n5. [bold dark_violet]Drink Alcohol[/bold dark_violet]\n6. [bold dark_violet]Smoke[/bold dark_violet]\n")
        #action_taken = 0
        #action_taken = IntPrompt.ask("Choose from 1 to 6", default=sa)
        action_taken = sa #automatic
        self.last_action = action_taken #automatic
            
        #update agent brain chemicals after action taken
        if(action_taken == 0): #Do Nothing/Sleep
            #1. update internal sensors or observations of the environment

            ## Update internal state upon consumption  
            interS = self.internal_state + (self.outcome[action_taken] * self.outcomeAbsorptionRatio) - self.outcomeDegradationRate * (self.internal_state - self.inStateLowerBound)
            if interS < self.inStateLowerBound:
                interS = self.inStateLowerBound    
            self.internal_state = interS

            ## Update homeostatic setpoint
            optInS = self.setpoint_S + self.outcome[action_taken]  * self.setpointShiftRate - self.setpointRecoveryRate
            if optInS < self.optimalInStateLowerBound:
                optInS = self.optimalInStateLowerBound
            if optInS > self.optimalInStateUpperBound:
                optInS = self.optimalInStateUpperBound
            self.setpoint_S = optInS

        elif((action_taken != 0) or (action_taken != -1)): #In-Active Lever/Active Lever #Binge on Internet, Work, Exercise, Socialise, Drink Alcohol, Smoke
            #1. determine the next state that the agent fell into from taking action
            #n/a
            
            #2. get Non-Homeostatic reward e.g energy cost or fatigue of doing an action
                ## if the agent is not sleeping then it is doing something that costs energy
                #nonHomeoRew = -self.fatigue
            self.estimatedNonHomeostaticReward = (1.0 - self.updateRewardRate) * self.estimatedNonHomeostaticReward + self.updateRewardRate * (self.fatigue)
            
            #3. get Homeostatically-regulated Reward of doing action (drive reduction)
            d1 = math.pow(math.fabs(math.pow(self.setpoint_S - self.internal_state, self.n*1.0)),(1.0/self.m))
            d2 = math.pow(math.fabs(math.pow(self.setpoint_S - self.internal_state - self.outcome[action_taken], self.n*1.0)),(1.0/self.m))
                #HomeoRew = d1 - d2
            self.driveReductionReward = (1.0 - self.updateOutcomeRate) * self.driveReductionReward + self.updateOutcomeRate * (d1 - d2)
            
            #4. Now calculate agent reward
            #self.reward_received = values[action] +  transitionProb * ( self.driveReductionReward + self.estimatedNonHomeostaticReward )
            #self.reward_received = qValue +  transitionProb * ( self.driveReductionReward + self.estimatedNonHomeostaticReward )
            self.reward_received = self.driveReductionReward + self.estimatedNonHomeostaticReward
            #5. update estimated next state
            # n/a
            
            #6. update internal sensors or observations of the environment
            
            ## Update internal state upon consumption
            self.outcomeBuffer = self.outcomeBuffer + self.outcome[action_taken]   
            interS = self.internal_state + (self.outcomeBuffer * self.outcomeAbsorptionRatio) - self.outcomeDegradationRate * (self.internal_state - self.inStateLowerBound)
            if interS < self.inStateLowerBound:
                interS = self.inStateLowerBound    
            self.internal_state = interS

            ## Update homeostatic setpoint
            optInS = self.setpoint_S + (self.outcome[action_taken]  * self.setpointShiftRate) - self.setpointRecoveryRate
            if optInS < self.optimalInStateLowerBound:
                optInS = self.optimalInStateLowerBound
            if optInS > self.optimalInStateUpperBound:
                optInS = self.optimalInStateUpperBound
            self.setpoint_S = optInS 

            ## Update outcome buffer
            self.outcomeBuffer = self.outcomeBuffer * (1 - self.outcomeAbsorptionRatio)

        elif(action_taken == -1):#quit
            return True   #end simulation
        
        clear_output(wait=True)
        print("** Current Time: [bold dark_violet]" + str(round((1/900)*(self.current_epoch),2)) + 
              "[/bold dark_violet] hrs, Epoch Left: [bold dark_violet]" + str(round((1/900)*(epochs_left),2)) + 
              "[/bold dark_violet] hrs, **\n[bold dark_green]Last Action: " + str(suggested_action) + 
              "[/bold dark_green],\n[bold dark_green]Current Homeostatic Variable: " + str(round(self.internal_state, 4)) + 
              "[/bold dark_green], [bold dark_green]Current Homeostatic Setpoint: " + str(round(self.setpoint_S, 4)) +
              "[/bold dark_green],\n [bold dark_green]Reward Received: " + str(round(self.reward_received,2)) + 
              "[/bold dark_green], [bold dark_green]Total Score: " + str(round(self.total_reward[-1],2)) + 
              "[/bold dark_green],\n [bold dark_green]Q-Values: ")# + str(qValues[0]) + 
              #"[/bold dark_green],\n [bold dark_green]T-Probs: " + str(transitionProbs[0]) + "[/bold dark_green]")
        #self.df.loc[self.current_epoch] = [self.current_epoch , suggested_action, qValues[0][0], qValues[0][1], qValues[0][2], transitionProbs[0][0], transitionProbs[0][1], transitionProbs[0][2], self.epochs_inactive, self.internal_state, self.setpoint_S, self.reward_received, self.total_reward[-1]]
    
        #check if this is the last round otherwise continue
        if(self.current_epoch >= self.total_epochs):
            return True   #end simulation
        else:    
            # Updating the last time from the agent to the end time (goal)
            self.current_epoch += 1  #update to next day interval

            #set time when active lever outcome 50 goes to zero
            if(self.epochs_inactive == 4):#lever is active
                self.epochs_inactive = 0 #off for 16 seconds then on the 20th second activate
                self.outcome_to_disable = self.outcome #remember what values were
                self.outcome = [0] * len(self.outcome)
            else:
                self.outcome = self.outcome_to_disable #restore outcome values
                self.epochs_inactive += 1 #count down by 4secs
                
            return False

    def clear(self): 
        """
        This function was taken from https://www.geeksforgeeks.org/clear-screen-python/ to
        allow the terminal to be cleared when changing menus or showing the user important
        messages. It checks what operating system is being used and uses the correct 
        clearing command.
        """
        # for windows 
        if name == 'nt': 
            _ = system('cls') 

        # for mac and linux(here, os.name is 'posix')
        else: 
            _ = system('clear')

In [22]:
if __name__ == "__main__":
    env = Simulation() 

IndexError: invalid index to scalar variable.

# Generate Dataset
## View Synthetic Dataset

In [18]:
#env.df.head(250)
#env.df.iloc[100:150] #when hit is taken
env.df.iloc[500:550]

,epoch,action,q_value_0,q_value_1,q_value_2,t_prob_0,t_prob_1,t_prob_2,epochs_inactive,internal_variable,homeostatic_setpoint,reward,score
500,500,Do Nothing,-0.000017,-24.261391,-17.384188,1.0,0.0,0.0,4,0.0,199.91984,0.0,0.0
501,501,Do Nothing,-0.000018,-24.261374,-17.384176,1.0,0.0,0.0,0,0.0,199.91968,0.0,0.0
502,502,Do Nothing,-0.000021,-24.261353,-17.384159,1.0,0.0,0.0,1,0.0,199.91952,0.0,0.0
503,503,Do Nothing,-0.000025,-24.261337,-17.384146,1.0,0.0,0.0,2,0.0,199.91936,0.0,0.0
504,504,Do Nothing,-0.000037,-24.261318,-17.384132,1.0,0.0,0.0,3,0.0,199.91920,0.0,0.0
505,505,Do Nothing,-0.000046,-24.261299,-17.384119,1.0,0.0,0.0,4,0.0,199.91904,0.0,0.0
506,506,Do Nothing,-0.000058,-24.261284,-17.384106,1.0,0.0,0.0,0,0.0,199.91888,0.0,0.0
507,507,Do Nothing,-0.000065,-24.261265,-17.384090,1.0,0.0,0.0,1,0.0,199.91872,0.0,0.0
508,508,Do Nothing,-0.000074,-24.261246,-17.384077,1.0,0.0,0.0,2,0.0,199.91856,0.0,0.0
509,509,Do Nothing,-0.000076,-24.261227,-17.384066,1.0,0.0,0.0,3,0.0,199.91840,0.0,0.0


## Export Dataset

In [19]:
x = datetime.datetime.now()
file_name = str(x.year) + "_" + str(x.month) + "_" + str(x.day) + "_" + str(x.strftime("%H")) + "_" + str(x.strftime("%M")) + "_" + str(x.strftime("%S")) + ".csv"
env.df.to_csv(file_name, index=False, encoding='utf-8')